In this Notebook I want to compare learnability of networks by different set of samples (some samples from diagonal frames and some samples from outer frames)

In [1]:
import os
import sys

model_path = "/Users/neda/HiCPlus_pytorch/src/models"
sys.path.insert(0, model_path)
import model

import numpy as np
import matplotlib.pyplot as plt
import pickle
import gzip
from torch.utils import data
import torch
import torch.optim as optim
from torch.autograd import Variable
from time import gmtime, strftime
import torch.nn as nn
from scipy.stats.stats import pearsonr
import argparse
use_gpu = 0
down_sample_ratio = 16
epochs = 50
HiC_max_value = 100 # ?????
batch_size = 256

In [2]:
indices = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878_primary/10kb_resolution/chr1-17-index.npy", "r")
indices = indices.astype("int64")
def d_indices(d):
    return np.where(indices[:,1] + d == indices[:,2])
def corr_highVSlow(index,data1,data2):
    return pearsonr(data1[index,0,:,:].flatten(),data2[index,0,:,:].flatten())[0]


In [3]:
# defining training data
# shift size indicate location of frames responding to matrix diagonal
get_minimum = 0
shift_size = 50
low_resolution_samples = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878_primary/10kb_resolution/chr1-17(down16)(rep2).npy", "r").astype(np.float32) * down_sample_ratio
low_resolution_samples = np.expand_dims(low_resolution_samples, axis=1)
high_resolution_samples = np.load("/Users/neda/HiCPlus_pytorch/data/divided-data/GM12878_primary/10kb_resolution/chr1-17.npy", "r").astype(np.float32)
high_resolution_samples = np.expand_dims(high_resolution_samples, axis=1)
high_resolution_samples = high_resolution_samples[d_indices(shift_size)[0],:,:,:]
low_resolution_samples = low_resolution_samples[d_indices(shift_size)[0],:,:,:]
if get_minimum == 1:
    high_resolution_samples = np.minimum(high_resolution_samples, HiC_max_value)
    low_resolution_samples = np.minimum(low_resolution_samples, HiC_max_value)


sample_size = high_resolution_samples.shape[-1]
half_padding = int(model.half_padding)
num_samples = low_resolution_samples.shape[0]

"""
lowres_set = torch.from_numpy(low_resolution_samples[39*256:40*256,])
hires_set = torch.from_numpy(high_resolution_samples[39*256:40*256,])
print("high and low loss: ", _loss(Variable(lowres_set), Variable(hires_set)).item())
zero_data = torch.from_numpy(np.zeros((256,1,sample_size,sample_size), dtype = 'float32'))
print("high and zero loss: ", _loss(Variable(zero_data), Variable(hires_set)).item())
"""

high_resolution_samples = high_resolution_samples[:,:,half_padding:(sample_size-half_padding),half_padding:(sample_size-half_padding)]
lowres_set = data.TensorDataset(torch.from_numpy(low_resolution_samples), torch.from_numpy(np.zeros(low_resolution_samples.shape[0])))
lowres_loader = torch.utils.data.DataLoader(lowres_set, batch_size=batch_size, shuffle=False)

hires_set = data.TensorDataset(torch.from_numpy(high_resolution_samples), torch.from_numpy(np.zeros(high_resolution_samples.shape[0])))
hires_loader = torch.utils.data.DataLoader(hires_set, batch_size=batch_size, shuffle=False)


for t in range(2):
    # defining network
    Net = model.Net(40, 28)
    if use_gpu:
        Net = Net.cuda()

    optimizer = optim.SGD(Net.parameters(), lr = 0.00001)
    _loss = nn.MSELoss()
    Net.train()
    running_loss = 0.0
    losslist = []
    for epoch in range(0, epochs):
        # iterate over two lists and their indices using enumerate together with zip
        # lowres_loader is list of batches
        for i, (v1, v2) in enumerate(zip(lowres_loader, hires_loader)):
            # probably it is for skipping last incomplete batch
            if (i == len(lowres_loader) - 1):
                continue


            # v1 is list with length = 2. v1[0] is data tensor so with shape 256*1*40*40. v1[1] is vector of 256 zeros because pf line 85 but what's the reason?
            _lowRes, _ = v1
            _highRes, _ = v2
            # print "_lowres:", _lowRes, "\n shape: ", _lowRes.shape

            _lowRes = Variable(_lowRes)
            _highRes = Variable(_highRes)


            if use_gpu:
                _lowRes = _lowRes.cuda()
                _highRes = _highRes.cuda()
            optimizer.zero_grad()
            y_prediction = Net(_lowRes)
            loss = _loss(y_prediction, _highRes)
            loss.backward()
            optimizer.step()
            #print(loss.item())
            running_loss += loss.item()
        print ('-------', i, epoch, running_loss/i, strftime("%Y-%m-%d %H:%M:%S", gmtime()))
        losslist.append(running_loss/i)
        running_loss = 0.0
    globals()["Net" + str(t)] = Net

------- 40 0 124.40880613327026 2019-07-08 19:37:21
------- 40 1 75.10643544197083 2019-07-08 19:37:23
------- 40 2 70.84276552200318 2019-07-08 19:37:25
------- 40 3 67.59781122207642 2019-07-08 19:37:27
------- 40 4 64.9930793762207 2019-07-08 19:37:29
------- 40 5 62.830317687988284 2019-07-08 19:37:32
------- 40 6 60.99898591041565 2019-07-08 19:37:34
------- 40 7 59.4310221195221 2019-07-08 19:37:36
------- 40 8 58.08386068344116 2019-07-08 19:37:38
------- 40 9 56.91957378387451 2019-07-08 19:37:40
------- 40 10 55.90814266204834 2019-07-08 19:37:43
------- 40 11 55.023833465576175 2019-07-08 19:37:45
------- 40 12 54.24432578086853 2019-07-08 19:37:47
------- 40 13 53.549905729293826 2019-07-08 19:37:49
------- 40 14 52.926847505569455 2019-07-08 19:37:52
------- 40 15 52.36129431724548 2019-07-08 19:37:54
------- 40 16 51.84192886352539 2019-07-08 19:37:56
------- 40 17 51.36294407844544 2019-07-08 19:37:59
------- 40 18 50.91817355155945 2019-07-08 19:38:03
------- 40 19 50.50

In [ ]:
temp_Net = Net0
lowres_set = torch.from_numpy(low_resolution_samples)
hires_set = torch.from_numpy(high_resolution_samples)
loss_list = []
for alpha in np.arange(0,1.01,0.01):
    
    for (temp_param, param1, param2) in zip(temp_Net.parameters(), Net0.parameters(), Net1.parameters()):
        temp_param.data = (alpha * param1.data) + ((1 - alpha) * param2.data)
    y_prediction = temp_Net(Variable(lowres_set))
    loss_list.append(_loss(y_prediction, Variable(hires_set)))


In [26]:
plt.plot(loss_list)
plt.axis(np.arange(0,1.001,0.001))
plt.show()


array([0.   , 0.001, 0.002, ..., 0.998, 0.999, 1.   ])

In [ ]:
# mean of correlation based on location of frames responding to diagonal of matrix
mean_int = {}
for i in range(-200,201,25):
    mean_int[i] = []
    for j in d_indices(i)[0]:
        mean_int[i].append(np.mean(low_resolution_samples[j,]))  
corr_list = {}
for i in range(-200,201,25):
    corr_list[i] = []
    for j in d_indices(i)[0]:
        if np.sum(low_resolution_samples[j,:,:]) != 0 and np.sum(high_resolution_samples[j,:,:]) != 0:
            corr_list[i].append(pearsonr(low_resolution_samples[j,0,:,:].flatten(),high_resolution_samples[j,0,:,:].flatten())[0]) 
mean_corr_list = [np.mean(corr_list[i]) for i in range(-200,201,25)]
import matplotlib.pyplot as plt
plt.scatter(range(-200,201,25), mean_corr_list)
plt.show()

In [16]:
for param in temp_Net.parameters():
  print(param.data)

tensor([[[[0.9636, 0.9208, 0.8555, 0.9137, 1.0335, 0.9685, 0.9639, 0.9570,
           0.9550],
          [0.8919, 0.9170, 0.9635, 0.9646, 0.9431, 0.9507, 1.0276, 0.9949,
           0.8679],
          [0.9759, 0.9762, 0.9202, 1.0667, 1.1664, 0.9842, 0.9624, 0.9654,
           0.9873],
          [0.9927, 0.9429, 1.1405, 1.1066, 1.3203, 1.1500, 1.0649, 1.0762,
           1.0207],
          [0.9871, 1.0559, 0.9653, 1.0166, 1.1858, 1.0775, 1.0898, 0.8916,
           0.9334],
          [0.9301, 0.9730, 0.9562, 0.9854, 1.0957, 1.0161, 1.0278, 0.9283,
           0.8892],
          [0.9084, 0.9565, 0.9525, 0.9693, 1.0521, 0.9755, 0.8908, 0.9872,
           0.9022],
          [0.9701, 0.9328, 1.0520, 0.9451, 0.9412, 0.9778, 0.9854, 0.8601,
           0.9632],
          [0.9036, 0.8899, 0.9923, 0.9370, 1.0664, 0.9955, 0.9547, 0.9384,
           0.8331]]],


        [[[0.9574, 0.9467, 1.0655, 1.0429, 1.1367, 0.9073, 1.0035, 1.0173,
           0.9325],
          [1.0506, 1.0344, 0.9583, 1.0991, 1.1